# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Embedded-in-Jupyter-notebook" data-toc-modified-id="Embedded-in-Jupyter-notebook-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Embedded in Jupyter notebook</a></div><div class="lev1 toc-item"><a href="#Using-GTK" data-toc-modified-id="Using-GTK-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Using GTK</a></div><div class="lev2 toc-item"><a href="#Example-of-how-to-compress-bytes-(e.g.,-JSON)-to-bzip2" data-toc-modified-id="Example-of-how-to-compress-bytes-(e.g.,-JSON)-to-bzip2-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Example of how to compress bytes (e.g., JSON) to bzip2</a></div>

# Embedded in Jupyter notebook

In [ ]:
%matplotlib notebook

import matplotlib as mpl
import matplotlib.pyplot as plt

import time
import threading

import ipywidgets as ipw
import numpy as np
import pandas as pd


fig, axis = plt.subplots()

axis._get_lines()
stop_event = threading.Event()
np.random.seed(0)
data = []


def _draw():
    while True:
        if data:
            pd.Series(np.concatenate(data)).plot(ax=axis)
            fig.canvas.show()
        data.append(np.random.rand(10))
        if stop_event.wait(.5):
            break
    stop_event.clear()
    start.disabled = False
    stop.disabled = True
        
    
def _start(*args):
    start.disabled = True
    stop.disabled = False
    thread = threading.Thread(target=_draw)
    thread.daemon = True
    thread.start()

start = ipw.Button(description='Start')
start.on_click(_start)
stop = ipw.Button(description='Stop')
stop.on_click(lambda *args: stop_event.set())
clear = ipw.Button(description='Clear')
def _clear(*args):
    axis.cla()
    for i in xrange(len(data)):
        data.pop()
clear.on_click(_clear)

widget = ipw.HBox([start, stop, clear])
widget

------------------------------------------------------------------------

# Using GTK

In [ ]:
import gtk
import gobject
import threading
import datetime as dt

import matplotlib as mpl
import matplotlib.style
import numpy as np
import pandas as pd

from mr_box_peripheral_board.ui.gtk.streaming_plot import StreamingPlot


def _generate_data(stop_event, data_ready, data):
    delta_t = dt.timedelta(seconds=.1)
    samples_per_plot = 5

    while True:
        time_0 = dt.datetime.now()
        values_i = np.random.rand(samples_per_plot)
        absolute_times_i = pd.Series([time_0 + i * delta_t
                                      for i in xrange(len(values_i))])
        data_i = pd.Series(values_i, index=absolute_times_i)
        data.append(data_i)
        data_ready.set()
        if stop_event.wait(samples_per_plot *
                           delta_t.total_seconds()):
            break
        
with mpl.style.context('seaborn',
                       {'image.cmap': 'gray',
                        'image.interpolation' : 'none'}):
    win = gtk.Window()
    win.set_default_size(800, 600)
    view = StreamingPlot(data_func=_generate_data)
    win.add(view.widget)
    win.connect('check-resize', lambda *args: view.on_resize())
    win.set_position(gtk.WIN_POS_MOUSE)
    win.show_all()
    view.fig.tight_layout()
    win.connect('destroy', gtk.main_quit)
    gobject.idle_add(view.start)
    
    def auto_close(*args):
        if not view.stop_event.is_set():
            # User did not explicitly pause the measurement.  Automatically
            # close the measurement and continue.
            win.destroy()
    gobject.timeout_add(5000, auto_close)
    
    measurement_complete = threading.Event()
    
    view.widget.connect('destroy', lambda *args: measurement_complete.set())

    gtk.gdk.threads_init()
    gtk.gdk.threads_enter()
    gtk.main()
    gtk.gdk.threads_leave()
    
    print measurement_complete.wait()

## Example of how to compress bytes (e.g., JSON) to bzip2

In [ ]:
from IPython.display import display
import bz2


data = pd.concat(view.data)
data_json = data.to_json()
data_json_bz2 = bz2.compress(data_json)
data_from_json = pd.read_json(bz2.decompress(data_json_bz2), typ='series')
len(data_json), len(data_json_bz2)